# 개정 크롤링
### 조건 별 크롤링
- 불러오는 데이터 : 1차 크롤링을 바탕으로 확인한 데이터들
- 검색했을 때 첫번째 나오는 것들인 경우는 이전 방법대로 정상적으로 진행
- 두번째 이후 나온 것들은 나온 것들끼리 묶어서 진행
    - 포맷팅 이용해서 크롤링해보기
- 검색했는데 나오지 않는 것들은 제외
- 예상 한 300개 정도 생략될 것임 -> 나중에 추가로 찾을 수 있다면 찾아보는 것으로?
- 이후에 이를 중심으로 리뷰를 분석해보고, 협업 필터링을 중심으로 추천시스템 구현

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.keys import Keys
import urllib

In [2]:
# 처리 완료되면 바꿔줘야함
df=pd.read_csv('죽기전에 가봐야할 101가지 여행지.csv')
df

,여행지
0,경복궁
1,삼청동 거리
2,인사동
3,조계사
4,청와대
...,...
994,쇠소깍
995,백조일손지묘
996,휴애리 자연생활공원
997,안덕계곡


In [3]:
df=pd.read_csv('여행지 명칭 확인_통합본.csv')
df

,여행지,일치여부,다른 칸에 존재여부
0,경복궁,O,NaN
1,삼청동 거리,O,NaN
2,인사동,O,NaN
3,조계사,O,NaN
4,청와대,O,NaN
...,...,...,...
999,쇠소깍,O,NaN
1000,백조일손지묘,X,X
1001,휴애리 자연생활공원,O,NaN
1002,안덕계곡,X,X


In [4]:
df_c=df[df['일치여부']=='O']
df_c=df_c[['여행지','일치여부']]
df_c.head()

,여행지,일치여부
0,경복궁,O
1,삼청동 거리,O
2,인사동,O
3,조계사,O
4,청와대,O


In [5]:
df_x=df[df['일치여부']=='X']
df_x

,여행지,일치여부,다른 칸에 존재여부
6,이화장,X,X
11,대학로,X,X
24,정동길,X,X
36,이슬람사원,X,X
45,최순우 옛집,X,X
...,...,...,...
994,추사 거적지,X,X
995,정석항공관,X,X
997,제주 조각공원,X,X
1000,백조일손지묘,X,X


In [6]:
df_xx=df_x[df_x['다른 칸에 존재여부']!='X']
df_xx

,여행지,일치여부,다른 칸에 존재여부
51,월드컵공원,X,2
53,대한민국 국회의사당과 헌정기념관,X,3
66,암사동 선사주거지,X,2
69,화성,X,2
70,안양 예술공원,X,4
81,부천 활박물관과 궁도장,X,2
84,대부도,X,2
94,국립현대미술관(과천관),X,2
120,파주 프로방스,X,8
123,설봉공원,X,4


In [7]:
df_xx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 51 to 991
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   여행지         55 non-null     object
 1   일치여부        55 non-null     object
 2   다른 칸에 존재여부  55 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [8]:
df_x[df_x['다른 칸에 존재여부']=='X'].count()

여행지           301
일치여부          301
다른 칸에 존재여부    301
dtype: int64

In [13]:
df_c[:20]

,여행지,일치여부
0,경복궁,O
1,삼청동 거리,O
2,인사동,O
3,조계사,O
4,청와대,O
5,보신각,O
7,국립민속박물관,O
8,사직단,O
9,경교장,O
10,탑골공원,O


In [10]:
df_xx

,여행지,일치여부,다른 칸에 존재여부
51,월드컵공원,X,2
53,대한민국 국회의사당과 헌정기념관,X,3
66,암사동 선사주거지,X,2
69,화성,X,2
70,안양 예술공원,X,4
81,부천 활박물관과 궁도장,X,2
84,대부도,X,2
94,국립현대미술관(과천관),X,2
120,파주 프로방스,X,8
123,설봉공원,X,4


In [14]:
df_c.to_csv('정상적 리뷰여행지.csv',encoding='utf-8-sig',index=False)
df_xx.to_csv('다른칸에있는 리뷰여행지.csv',encoding='utf-8-sig',index=False)

In [16]:
df_c=pd.read_csv('정상적 리뷰여행지.csv')
df_xx=pd.read_csv('다른칸에있는 리뷰여행지.csv')

In [17]:
df_c

,여행지,일치여부
0,경복궁,O
1,삼청동 거리,O
2,인사동,O
3,조계사,O
4,청와대,O
...,...,...
608,알뜨르 비행장,O
609,이중섭 미술관,O
610,쇠소깍,O
611,휴애리 자연생활공원,O


In [18]:
df_xx

,여행지,일치여부,다른 칸에 존재여부
0,월드컵공원,X,2
1,대한민국 국회의사당과 헌정기념관,X,3
2,암사동 선사주거지,X,2
3,화성,X,2
4,안양 예술공원,X,4
5,부천 활박물관과 궁도장,X,2
6,대부도,X,2
7,국립현대미술관(과천관),X,2
8,파주 프로방스,X,8
9,설봉공원,X,4


2번째 : #BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(2) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div

3번째 : #BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div

 driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(%d) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div').click()

In [26]:
# 문제가 없는 것들 먼저
options=webdriver.ChromeOptions()
options.add_argument('user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

basepath='C:/Users/이신행/OneDrive/바탕 화면/chromedriver_win32(2)/'
driver=webdriver.Chrome()
url='https://www.tripadvisor.co.kr/'

reviews=[]
for i in range(len(df_c)):
    driver.get(url)
    search=df_c.여행지[i]
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(search)
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    try: 
        # 약간 문제인 부분
        driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(1) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div').click()

        driver.switch_to.window(driver.window_handles[-1])
        soup=BeautifulSoup(driver.page_source,'html.parser')
        mydata=soup.find_all('span','NejBf')
        for item in mydata:
            reviews.append(search + '-->' + item.get_text().strip())
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.back()
        time.sleep(1)
    except:
        pass

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.67)


In [32]:
# 문제가 없는 것들 먼저 (오류날때 난 시점부터 돌리기)
options=webdriver.ChromeOptions()
options.add_argument('user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

basepath='C:/Users/이신행/OneDrive/바탕 화면/chromedriver_win32(2)/'
driver=webdriver.Chrome()
url='https://www.tripadvisor.co.kr/'

for i in range(452, len(df_c)):
    driver.get(url)
    search=df_c.여행지[i]
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(search)
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    try: 
        # 약간 문제인 부분
        driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(1) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div').click()

        driver.switch_to.window(driver.window_handles[-1])
        soup=BeautifulSoup(driver.page_source,'html.parser')
        mydata=soup.find_all('span','NejBf')
        for item in mydata:
            reviews.append(search + '-->' + item.get_text().strip())
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.back()
        time.sleep(1)
    except:
        pass

In [33]:
reviews[-5:]

['제주 감귤박물관-->감귤의 제배 과정 효능 방법등에 대해 나와 있고 종류또한  다양합니다. 아이들과 방문한다면 좋은 공부가 될수도 있습니다.',
 '제주 감귤박물관-->감귤박물관',
 '제주 감귤박물관-->생각보다 아주 크게 있음. 감귤을 수확하는 모든 과정을 볼 수 있으며 체험도 할 수 있어 재밌음.',
 '제주 감귤박물관-->신나는 감귤 축제',
 '제주 감귤박물관-->감귤 축제 기간에 방문하니 감귤 수확 체험이 평소 6000원이던 것이 3000원으로 할인해주고 직원들이 수확하는 방법도 자세히 알려주시고 목장갑, 가위도 다 빌려주시기 때문에 아이들과 체험하기 좋았다.박물관도 생각보다 크고, 오래되긴 했지만 깨끗하게 관리되고 있으며, 그린하우스 내에 세계 여러나라의 작고 큰 귤나무가 다양하게 있어 좋았다.밑에 리뷰에 있는 것처럼 애월이 아니라(애월은 제주시에 있음) 서귀포 효돈에 위치하고 있다. 주차장도 꽤 큰 편이다. 감귤 과즐이 맛있어서 한 번 사먹어 보기를 권한다.']

In [31]:
df_c[df_c['여행지']=='괴시리 전통마을']

,여행지,일치여부
451,괴시리 전통마을,O


In [34]:
# 포맷팅 해서 적용하는 코드 (df_xx사용)
options=webdriver.ChromeOptions()
options.add_argument('user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

basepath='C:/Users/이신행/OneDrive/바탕 화면/chromedriver_win32(1)/'
driver=webdriver.Chrome()
url='https://www.tripadvisor.co.kr/'

for i in range(len(df_xx)):
    driver.get(url)
    search=df_xx.여행지[0]
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(search)
    driver.find_element_by_css_selector('#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    
    try : 
        if df['다른 칸에 존재여부']==2:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(2)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==3:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(3)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==4:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(4)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==5:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(5)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==6:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(6)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==7:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(7)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==8:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(8)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==9:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(9)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        elif df['다른 칸에 존재여부']==10:
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child({0}) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div'.format(10)).click()
            driver.switch_to.window(driver.window_handles[-1])
            soup=BeautifulSoup(driver.page_source,'html.parser')
            mydata=soup.find_all('span','NejBf')
            for item in mydata:
                reviews.append(search + '-->' + item.get_text().strip())
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.back()
            time.sleep(1)
        else:
            pass
    except:
        pass

KeyboardInterrupt: 

In [ ]:
# 명주사 지워야됨

In [46]:
titles=[]
contents=[]
for i in range(0,len(reviews),2):
    titles.append(reviews[i])
for j in range(1,len(reviews),2):
    contents.append(reviews[j])

In [47]:
len(titles)

4702

In [48]:
len(contents)

4701

In [43]:
titles[-20:]

['휴애리 자연생활공원-->방문했을때 너무 더웠지만 다양한 꽃을 볼 수 있어서 좋았어요. 좀 시원해지면 커플에게도 추천할만한 곳인 것 같다고 생각합니다~!!',
 '휴애리 자연생활공원-->시기적으로 동백이 지는무렵방문하여(12월23일) 그런지 잎이 많이 지고, 아직은 사진찍을 만한 곳도 많지만, 입장료가 좀 아까웠다. 성인 13000원 ! 인터넷구매하여 10%할인받았지만, 그래도 2시간정도 즐기다 가기엔....',
 '휴애리 자연생활공원-->현장 예매는 13,000원입니다. 인터넷 예약은 더 저렴한걸로 알고 있으나, 3시간인가 전에 해야 발권 가능하다더군요. 사진찍기 좋아하시는 분들이 가면 인생샷 건지기 좋을듯요. 제가 갔을땐 동백꽃이 주였습니다^^',
 '휴애리 자연생활공원-->수국축제 최고입니다.수국 좋아하면 휴애리 강추 입니다다양한 동물도 직접 볼수 있고포토존이 많아 연인들 스냅사진 찍기 좋아요',
 '휴애리 자연생활공원-->아이가 있는 집은 강추.여러가지 동식물이 있고, 먹이도 줄 수 있음.가을이라 핑크뮬리가 예쁨.흑돼지, 거위쇼도 하는데 아이들이 정말 좋아함. 비가 와서 급하게 다니느라 제대로 못 봐서 아쉬움.',
 '휴애리 자연생활공원-->휴애리 하면 흑돼지쇼만 생각했는데동백, 매화, 갖춰놓은 정원이 참으로 좋았어요.사진 찍기에도 테이블도 많지는 않지만 적당히 있고 푸드트럭도 있고 해서 먹을 거리도 있고,소풍 나오기 좋은 것 같아요.흑돼지쇼는 매시간 정각에 한번씩 하는데 볼만해요.흑돼지쇼에 이어 거위쇼까지 하네요,어른도 흥미로운데 아이들에게는 정말 유익한 시간이 될 것 같아요.역시 가족단위가 많네요~~',
 '휴애리 자연생활공원-->귀여운 아기동물들도 보고, 말타기 체험도 할수있고 포토존도 많고 솜사탕도 있어서 아이들이 정말 좋아했어요!',
 '휴애리 자연생활공원-->가족들과 같이 가서 사진찍기 좋은곳 입니다.돼지쇼가 있는데 별건 없지만 아이들이 좋아합니다.겨울에 매화축제를 하는데 올해는 폭설로 늦어진다고 하네요.일단 가시면 시간을 먼저 보시고 돼지

In [52]:
df_1=pd.DataFrame({'리뷰제목':titles})
df_2=pd.DataFrame({'리뷰내용':contents})

In [53]:
df_1

,리뷰제목
0,경복궁-->대한민국의 역사
1,경복궁-->국민이 공감하는 장소
2,경복궁-->산책하기 좋은 경복궁
3,경복궁-->Good
4,경복궁-->가족단위로 방문하기 좋은곳
...,...
4697,제주 감귤박물관-->제주 감규의 역사에 대해 알수 있습니다.세게 감귤 전시과 및 열...
4698,제주 감귤박물관-->잘 만들어 놓았는데 관광객들이 오가기엔 조금 애매한가봅니다.서귀...
4699,제주 감귤박물관-->감귤의 제배 과정 효능 방법등에 대해 나와 있고 종류또한 다양...
4700,제주 감귤박물관-->생각보다 아주 크게 있음. 감귤을 수확하는 모든 과정을 볼 수 ...


In [55]:
df_1['관광지']=df_1.리뷰제목.str.split('-->').str[0]
df_1['리뷰제목']=df_1.리뷰제목.str.split('-->').str[1]

In [57]:
df_1=df_1[['관광지','리뷰제목']]
df_1

,관광지,리뷰제목
0,경복궁,대한민국의 역사
1,경복궁,국민이 공감하는 장소
2,경복궁,산책하기 좋은 경복궁
3,경복궁,Good
4,경복궁,가족단위로 방문하기 좋은곳
...,...,...
4697,제주 감귤박물관,"제주 감규의 역사에 대해 알수 있습니다.세게 감귤 전시과 및 열대 식물 온실, 족욕..."
4698,제주 감귤박물관,잘 만들어 놓았는데 관광객들이 오가기엔 조금 애매한가봅니다.서귀포시 직영관광지인 ...
4699,제주 감귤박물관,감귤의 제배 과정 효능 방법등에 대해 나와 있고 종류또한 다양합니다. 아이들과 방...
4700,제주 감귤박물관,생각보다 아주 크게 있음. 감귤을 수확하는 모든 과정을 볼 수 있으며 체험도 할 수...


In [58]:
df_2

,리뷰내용
0,경복궁-->대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해야 되는 ...
1,경복궁-->경복궁은 국민들이 자주 찾는곳으로 작성자는 주말에 자주 가족들과 방문하고...
2,경복궁-->날씨 좋은 날 종종 산책하러 경복궁에 가는데 마음이 편온해지는 기분이라고...
3,경복궁-->Goooooood 다 좋습니다 다음에 또 오고 싶네요 근처 관광지도많고 ...
4,경복궁-->요새 더더욱 코로나로 인해 사람 방문이 적음. 두자녀 동반시 성인 입장무...
...,...
4696,제주 감귤박물관-->생각보다 사람이 없네요.
4697,"제주 감귤박물관-->한국 감귤의 역사, 세계 감귤 전시관,열대 식물 온실, 족욕체험..."
4698,제주 감귤박물관-->제주도의 역사에대해 나와있습니다.
4699,제주 감귤박물관-->감귤박물관


In [60]:
df_2['관광지']=df_2.리뷰내용.str.split('-->').str[0]
df_2['내용']=df_2.리뷰내용.str.split('-->').str[1]

In [62]:
df_2=df_2[['관광지','내용']]
df_2

,관광지,내용
0,경복궁,대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해야 되는 곳. 경복궁...
1,경복궁,경복궁은 국민들이 자주 찾는곳으로 작성자는 주말에 자주 가족들과 방문하고 있음.특히...
2,경복궁,날씨 좋은 날 종종 산책하러 경복궁에 가는데 마음이 편온해지는 기분이라고 할까요? ...
3,경복궁,Goooooood 다 좋습니다 다음에 또 오고 싶네요 근처 관광지도많고 먹을거리도많...
4,경복궁,요새 더더욱 코로나로 인해 사람 방문이 적음. 두자녀 동반시 성인 입장무료. 지금 ...
...,...,...
4696,제주 감귤박물관,생각보다 사람이 없네요.
4697,제주 감귤박물관,"한국 감귤의 역사, 세계 감귤 전시관,열대 식물 온실, 족욕체험도 무료"
4698,제주 감귤박물관,제주도의 역사에대해 나와있습니다.
4699,제주 감귤박물관,감귤박물관


In [63]:
df_1.to_csv('리뷰제목.csv',encoding='utf-8-sig',index=False)
df_2.to_csv('리뷰내용.csv',encoding='utf-8-sig',index=False)

In [45]:
contents[:]

['경복궁-->대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해야 되는 곳. 경복궁은 우리의 역사다. 넓은 경복궁을 산책할 수 있는 것은 언제나 행복이다.',
 '경복궁-->경복궁은 국민들이 자주 찾는곳으로 작성자는 주말에 자주 가족들과 방문하고 있음.특히 가족들과 방문시 옛고궁의 멋을 즐길수 있으며 쾌적한 공기와역사를 체험하고 힐링할수 있는곳임.',
 '경복궁-->날씨 좋은 날 종종 산책하러 경복궁에 가는데 마음이 편온해지는 기분이라고 할까요? 특히 봄이나 가을에 산책가는 걸 추천해요!',
 '경복궁-->Goooooood 다 좋습니다 다음에 또 오고 싶네요 근처 관광지도많고 먹을거리도많네요 서울올때마다 자주이용중인데 또오고싶습니다',
 '경복궁-->요새 더더욱 코로나로 인해 사람 방문이 적음. 두자녀 동반시 성인 입장무료. 지금 시기에 나들이 하기 딱 좋은곳. 지하철 광화문역이나 종각역이용 편리.',
 "경복궁-->조선 개국 4년째인 1395년에 처음으로 세운 으뜸 궁궐이다.'하늘이 내린 큰 복' 이라는 뜻으로 경복궁이라 이름지었다.가장 아름답고 웅장한 이 경복궁은 임진왜란, 일제강점기 2번에 걸쳐 일본의 악랄하고 잔인한 파괴를 당한 악연을 갖고있다.향원정은 보수 공사로 인해 볼 수 없지만 아름다운 경회루만으로도 충분한 경복궁이다.",
 '경복궁-->한국 건축물의 아름다움을 느낄 수 있는 곳이고, 한복을 입었을 때 입장료는 무료.주변에 한복 대여점이 많으니 한번쯤 대여해서 입어보는것도 나쁘지 않을듯',
 '경복궁-->조선의 성곽 안에는 박물관도 있고 경회루 등 볼것이 많다. 외국인들 및 연인들도 주변에서 한복 및 특수복장 빌려서 성을 둘러보며 데이트를 즐기기에 딱 좋다. 야간개장도 하니 잘 알아보고 올것 한국사람이라면 한번이상은 꼭와봤고 , 외국인들에게도 좋은 관광지라고 생각한다. 앞에 광화문 광장 청계천 뒤로 삼청동  이주변에 연계된 명소들도 많기 때문에 꼭 추천한다.',
 '경복궁-->대부분의 사람들이 여행을 간다면 그 나라의 유서 깊은 궁

In [36]:
data=pd.DataFrame({'리뷰제목':titles,'내용':contents})
data

ValueError: All arrays must be of the same length

In [ ]:
data.to_csv('여행지 리뷰 1차.csv',encoding='utf-8-sig',index=False)

In [ ]:
data['관광지']=data.리뷰제목.str.split('-->').str[0]
data['리뷰제목']=data.리뷰제목.str.split('-->').str[1]
data['내용']=data.내용.str.split('-->').str[1]

In [ ]:
data=data[['관광지','리뷰제목','내용']]
data

In [ ]:
data.to_csv('여행지 리뷰 2차.csv',encoding='utf-8-sig',index=False)

- 별도 실험용 코드

In [1]:
options=webdriver.ChromeOptions()
options.add_argument('user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

basepath='C:/Users/이신행/OneDrive/바탕 화면/chromedriver_win32(2)/'
driver=webdriver.Chrome()
url='https://www.tripadvisor.co.kr/Tourism-g294196-South_Korea-Vacations.html'

driver.get(url)
search=df.여행지[0]
driver.find_element_by_css_selector('#lithium-root > header > div > nav > div > div.fUFMI > div > div.weiIG.Z0.Wh.bZZEX > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(search)
driver.find_element_by_css_selector('#lithium-root > header > div > nav > div > div.fUFMI > div > div.weiIG.Z0.Wh.bZZEX > form > input.fhEMT._G.B-.z._J.Cj.R0').send_keys(Keys.ENTER)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(2)

soup=BeautifulSoup(driver.page_source,'html.parser')
mydata=soup.find_all('div','location-meta-block')
mydata[1].get_text().strip()

NameError: name 'webdriver' is not defined

In [8]:
mydata[1].get_text().strip()

'경복궁21건의 리뷰종로구 청진동 243, 서울, 대한민국종로구 청진동 243, 서울'